TEST LOADER FOR PHOIBOS 150 DATA

Typical structure 
/2020/01 January/Day 10/Raw Data/8874

Withing the folder of the scan number there is an info.txt and a scan
In THE AVG FOLDER there is a set of images, averaged over every acquisition cycle
IN THE RAW FOLDER there are single acquisition cycles (difference: in EPFL we added the "save every n cycle function"
this RAW folder also contains a "LUT.txt" file

In [ ]:
from importlib import reload
from specsanalyzer import SpecsAnalyzer
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
spa = SpecsAnalyzer()

import os
exampledatapath='../tests/data/dataEPFL/9159/'
filelist=os.listdir(exampledatapath)
print("example data path= ",exampledatapath)
#this folders contains the AVG folder with the data, an info.txt vector and a scan vector

#navigate to the image folder, for EPFL in the \AVG subfolder of the data
avgimagepath='../tests/data/dataEPFL/9159/AVG'
print("AVG images path= ",avgimagepath)
rawimagelist=os.listdir(avgimagepath)


In [ ]:
raw_image_name=os.path.join(avgimagepath,rawimagelist[0]) 
with open(raw_image_name) as file:
    tsv_data = np.loadtxt(file, delimiter='\t')
plt.figure()
h = plt.contourf(tsv_data)
plt.colorbar()
print("The shape of the raw data is: ",tsv_data.shape)
#plt.xlim(120,150)
#plt.ylim(150,200)


In [ ]:
from specsanalyzer import SpecsAnalyzer
reload(specsanalyzer)
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
spa = SpecsAnalyzer()
scanparameters=spa.return_config()

In [ ]:
#load the module and import functions for reading the parameter table from the calib2d file
import specsanalyzer.convert
reload(specsanalyzer.convert)   


#let's get all the functions to be tested
from specsanalyzer.convert import get_damatrix_fromcalib2d
from specsanalyzer.convert import get_rr_da
from specsanalyzer.convert import calculate_polynomial_coef_da
from specsanalyzer.convert import mcp_position_mm


In [ ]:
from specsanalyzer import SpecsAnalyzer
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

# call an instance of the class 
spa = SpecsAnalyzer()
# io function used to get the calib 2d file in the class
calib2d_dictionary=specsanalyzer.io.parse_calib2d_to_dict('../tests/data/dataEPFL/phoibosEPFL.calib2d')
# settings function used to get  configuration files  
config_file=specsanalyzer.settings.parse_config(config_dictionary)



#get the das  and the rr vector 
#rr_array, da_matrix=  specsanalyzer.convert.get_rr_da("WideAngleMode", config_file)

In [ ]:
#get the rr and the damatrix
rr, damatrix= get_rr_da("WideAngleMode",spa.return_config())

In [ ]:
x = [1e-5, 1000, 1e-1]

y = np.arccos(np.cos(x))

a=np.testing.assert_allclose(x, y, rtol=1e-5, atol=0)
print(a)

In [ ]:
# For the example scan we have the following settings:
ek=float(scanparameters["KineticEnergy"])
ep=float(scanparameters["PassEnergy"])
erange=scanparameters["eRange"]
arange=scanparameters["aRange"]
angles=np.linspace(-20,20,1024)
energies=np.linspace(ek+ep*erange[0],ek+ep*erange[1],2048)
ainner=15
energymesh,anglemesh=np.meshgrid(energies,angles)

testimage=mcp_position_mm(energymesh,anglemesh,scanparameters)

In [ ]:
from matplotlib import cm

In [ ]:
# For the example scan we have the following settings:
(ek_axis, angle_axis, angular_correction_matrix, e_correction,
         jacobian_determinant)=calculate_matrix_correction(scanparameters)

In [ ]:
from matplotlib import cm

plt.contourf(ek_axis,angle_axis,angular_correction_matrix)
plt.colorbar()

In [ ]:
plt.contourf(ek_axis,angle_axis,jacobian_determinant)
plt.colorbar()

In [ ]:
plt.plot(e_correction)

In [ ]:
h = plt.contourf(raw_data)
plt.colorbar()

In [ ]:
print(ek_axis.shape)
print(angle_axis.shape)
print(corrected_data.shape)
colorlevels=np.linspace(0,130000,100)
h = plt.contourf(ek_axis,angle_axis,corrected_data,levels=colorlevels)
plt.colorbar()
#plt.xlim(34,37)
#plt.ylim(-15,15)

In [ ]:
h = plt.contourf(angle_axis,ek_axis,corrected_data.transpose(),levels=colorlevels)
plt.colorbar()